In [12]:
%run read_data_1point5.ipynb

In [13]:
script_dir = os.path.dirname('__file__')
results_dir = os.path.join(script_dir, 'Indv_train/')
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)
    
#script_dir = os.path.dirname('__file__')
results_dir1 = os.path.join(script_dir, 'Indv_train_CDF/')
if not os.path.isdir(results_dir1):
    os.makedirs(results_dir1)
    
#script_dir = os.path.dirname('__file__')
results_dir2 = os.path.join(script_dir, 'Indv_train_count/')
if not os.path.isdir(results_dir2):
    os.makedirs(results_dir2)
    
#script_dir = os.path.dirname('__file__')
results_dir3 = os.path.join(script_dir, 'Indv_train_box/')
if not os.path.isdir(results_dir3):
    os.makedirs(results_dir3)

In [14]:
%run functionv2.ipynb

In [15]:
start_time = timeit.default_timer()

cal_u = copy.deepcopy(wtt_u)
cal_d = copy.deepcopy(wtt_d)


#f = open("Train_data_length.txt", "a")
for i in range(8):
    runtime_block1 = []
    for j in range(3,61):
        data3 = rt_names[i].iloc[:,[0,j]]
        #print(data3)
        for l in range(54):
            if (i>3):
                wttv = wtt_d.iloc[l,j]
                wtt_t = wtt_type_d.iloc[l,j]
            else:
                wttv = wtt_u.iloc[l,j]
                wtt_t = wtt_type_u.iloc[l,j]
                
                
            #Calculate runtime only from realised halt type data
            if (wtt_t!=stop_type[i]):
                continue
                
            #Getting runtime data    
            if (i>3):
                data1 = data3[data3['TrainMaskNo']==wtt_d.iloc[l,0]]
            else:
                data1 = data3[data3['TrainMaskNo']==wtt_u.iloc[l,0]]
            data0 = data1.iloc[:,1]
            original_data1 = [k for k in data0 if str(k) != 'nan']
            if (len(original_data1)<25):
                continue
            original_data = outlier(original_data1)
            #print(original_data)
            if (len(original_data)<150):
                data2 = clt_conversion_10_per_org(data0)
            else:
                data2 = clt_conversion_10_per_150_org(data0)
            data = outlier(data2)
            if (len(data)<25):
                #print(str(j-2)+'.'+str(l+1)+' '+str(plot_type[i])+"Less 20",file=f)
                runtime_block1.append(np.nan)
                continue
            #print(str(j-2)+'.'+str(l+1)+' '+str(plot_type[i])+"  Length of data=",len(data),file=f)
            t_dist = stats.t
            par_est = t_dist.fit(data)
            generated_data = t_dist.rvs(*par_est[:-2], loc=par_est[-2], scale=par_est[-1], size=len(data))
            rt_cal=0
            if (len(generated_data)>100):
                rt_cal = max(generated_data)+0.1*max(generated_data)
                rt_cal1 = str(rt_cal)
                n_decimal = rt_cal1.index('.')
                f_decimal = int(rt_cal1[n_decimal +1])
                if (f_decimal>=3):
                    rt_cal=math.ceil(rt_cal)
                else:
                    rt_cal=math.floor(rt_cal)
            else:
                rt_cal = max(generated_data)-0.1*max(generated_data)
                rt_cal1 = str(rt_cal)
                n_decimal = rt_cal1.index('.')
                f_decimal = int(rt_cal1[n_decimal +1])
                if (f_decimal>=5):
                    rt_cal=math.floor(rt_cal)
                else:
                    rt_cal=math.ceil(rt_cal)
            if (i>3):
                cal_d.iloc[l,j]=rt_cal
            else:
                cal_u.iloc[l,j]=rt_cal
            
  
elapsed = timeit.default_timer() - start_time
print(elapsed)

254.7654908679542


In [16]:
cal_d.to_csv('cal_d.csv')
cal_u.to_csv('cal_u.csv')